<a href="https://colab.research.google.com/github/yamadashamoji/tools/blob/main/009_2_PDF_extra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF
import fitz  # PyMuPDF
import ipywidgets as widgets
from IPython.display import display
from google.colab import files

# アップロードウィジェット
upload_btn = widgets.FileUpload(accept='.pdf', multiple=False)
display(upload_btn)

# 抽出ボタン
output = widgets.Output()
page_input = widgets.Text(description='ページ指定:', placeholder='例: 1,3-5（空欄なら全ページ）')
extract_btn = widgets.Button(description='テキスト抽出', button_style='success')
display(page_input, extract_btn, output)

def extract_text(change):
    output.clear_output()
    if not upload_btn.value:
        with output:
            print("PDFファイルをアップロードしてください。")
        return

    uploaded_file = list(upload_btn.value.values())[0]
    pdf_bytes = uploaded_file['content']
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")

    # ページ指定の解析
    selected_pages = []
    if page_input.value.strip():
        import re
        for part in page_input.value.split(','):
            if '-' in part:
                start, end = map(int, part.split('-'))
                selected_pages.extend(range(start-1, end))
            else:
                selected_pages.append(int(part)-1)
    else:
        selected_pages = list(range(len(doc)))

    text = ""
    for i in selected_pages:
        if 0 <= i < len(doc):
            text += f"--- ページ {i+1} ---\n"
            text += doc[i].get_text()
            text += "\n\n"

    with open("extracted_text.txt", "w", encoding="utf-8") as f:
        f.write(text)

    with output:
        print(text[:1000])  # 最初の1000文字だけ表示
        print("...（以下省略）")
        print("\nテキストを抽出しました。下記からダウンロード可能です。")

    files.download("extracted_text.txt")

extract_btn.on_click(extract_text)
